## Coding Method Explanations for Qiang

### Data Preparation

I use the code dataset.py from the Stanford Fake News codes, as I have not personally identified any differences in our pre-processing requirements to theirs.

### Building the Graph

Again, I almost exactly use the Stanford code generate_graphs.py. I have made the graph dynamic by creating an array that appends the collection of edges each time a new edge is formed. This means we have an array 'edges', which stores the current graph snapshot and an array 'dynamic_graph', which stores each timestamped version of 'edges'. 

I show the two adjusted functions below. My version of the Stanford code generate_graphs.py is simply titled graphs.py and is among the other documents I have sent you.

In [1]:
def generate_dynamic_graph(dumps_dir, dataset, timestamp):
    """
    :return: simple event graph, directed, unweighted, with edges created up to current timestamp
             dynamic graph, as a sequence of graph snapshots taken each time a new event node is added
    """
    nodes = {}
    edges = []
    dynamic_graph = []
    num_nodes = 0
    dataset_dir = os.path.join(dumps_dir, dataset)
    for data_point in twitter_tree_iterator():
        if data_point['time_out'] < timestamp:
            node_in, node_out = data_point['node_in'], data_point['node_out']
            if node_in not in nodes:
                nodes[node_in] = num_nodes
                num_nodes += 1
            if node_out not in nodes:
                nodes[node_out] = num_nodes
                num_nodes += 1
            edges.append([nodes[node_in], nodes[node_out]])
            dynamic_graph.append(edges)

    return edges, dynamic_graph

def generate_datum(dynamic_graph):
    """
    :return: datum of form (claim, c; list of engagements, S; dynamic graph, G)
    """
    claim = dynamic_graph[0][0][0]
    engagements = dynamic_graph[-1][1:]
    datum = [claim, engagements, dynamic_graph]
    
    return datum

### Encoder: Temporal Graph Sum 

In the document temporal_graph_sum.py, I attempt to implement equations (1) and (2) of the overleaf document. I think that equation (1) is more likely to be correctly implemented, but I am unsure how to include $\phi$, the time embedding:

In [3]:
def eq_1():
    h = []
    for i in dynamic_graph:
        h.append(np.zeros_like(dynamic_graph[i]))
    h_tilde = np.zeros_like(h)
    del h_tilde[0]
    h[0] = dynamic_graph[0]
    for i in h_tilde:
        for j in h_tilde[i]:
            h_tilde[i][j] = np.sum(np.concatenate(h[i][j],edges[j]))

I then attempt to implement the MLP of equation 2, such that the encoder roughly looks as follows:

In [5]:
import torch
class TGS_stack(torch.nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TGS_stack, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim  = hidden_dim
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()
    
    
    def node_embeddings(dynamic_graph):
        h = []
        for i in dynamic_graph:
            h.append(np.zeros_like(dynamic_graph[i]))
        h_tilde = np.zeros_like(h)
        del h_tilde[0]
        h[0] = dynamic_graph[0]
        for i in h_tilde:
            for j in h_tilde[i]:
                h_tilde[i][j] = np.sum(np.concatenate(h[i][j],edges[j]))
            
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output
            
        for i in h:
            for j in h[i]:
                h[i+1][j] = forward(np.concatenate(h[i][j],h_tilde[i][j]))
                
        return h
    
    def generate_hidden(h):
        hidden = []
        for i in h[-1]:
            hidden.append(h[-1][i])
            
        return hidden

This currently only uses a single-layer perceptron. I am unsure how to extend it to multi-layer.

### Claim Veracity Prediction

Here, I attempt to implement equations (14) and (15) of the overleaf document. I have spent lots of time reading the literature on graph learning algorithms as well as pytorch documentation, so I understand the concepts quite well, but given my lack of experience, I am still struggling with transferring that into the correct code. My very rough idea of how to execute this task is shown below and in veracity_prediction.py file I have sent you. 

In [7]:
class veracity(torch.nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(veracity, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim  = hidden_dim
        self.relu = torch.nn.ReLU()
        self.weight = torch.nn.Parameter(data=torch.randn(len(dynamic_graph[-1])), requires_grad=True)
        self.bias = torch.nn.Parameter(data=torch.Tensor(0,0), requires_grad=False)
        
    def forward(self,x):
        hidden = x*self.weight + self.bias
        relu = self.relu(hidden)
        soft = np.exp(relu - np.max(relu))
        output = soft/soft.sum()
        return output
    
    def compute_loss(self,output,veracity_label):
        y_1 = output[0]
        y_2 = output[1]
        loss = -veracity_label*np.log(y_2) - (1-veracity_label)*np.log(1 - y_1)
        return loss

I am particularly unsure how to initialise the learnable weight matrix and bias term. Perhaps it would be beneficial for you to point me along some more explicit lines if you do not think my implementations very appropriate. Or if you think I am along the right lines, I am happy to continue with these approaches.

### Further Work

I will use your codes for the Hawkes process method and implement the timestamp prediction and stance classification tasks.